In [1]:
import numpy as np
import pandas as pd
import json
from evaluator import EvalAIAnswerProcessor, TextVQAAccuracyEvaluator
import pickle
from functools import reduce

In [2]:
with open("model_preds.pickle", "rb") as f:
    model_preds = pickle.load(f)



In [33]:
obj_q = open("./subsets/val_obj_question.txt").read().split(",")
obj_q = [int(o) for o in obj_q]
pos_q = open("./subsets/val_pos_question.txt").read().split(",")
pos_q = [int(o) for o in pos_q]
text_q = open("./subsets/val_require_reading.txt").read().split(",")
text_q = [int(o) for o in text_q]

In [3]:
models = ["lorra", "m4c", "tap", "hotpot-lorra", "hotpot-without-mmt", "hotpot-without-object-label", "hotpot", "object", "ocr", "question"]
grade_books = ["whole", "pos", "obj"]

In [4]:
for g in grade_books:
    df = pd.read_csv("./gradebooks/grade_book_"+g+".csv", index_col=None).round(1)
    vc = [df["accu_"+m].value_counts().rename_axis("accu").reset_index(name=m) for m in models]
    dist = reduce(lambda x, y: pd.merge(x, y, on = 'accu', how="inner"), vc)
    dist = dist.sort_values(by=['accu'])
    dist.to_csv("accu_dist_"+g+".csv", index=False)

In [5]:
whole_grade_books = pd.read_csv("./gradebooks/grade_book_whole.csv")

In [6]:
max = whole_grade_books.iloc[:, 4:].max(axis = 1)
min = whole_grade_books.iloc[:, 4:].min(axis = 1)
sec_min = whole_grade_books.iloc[:,4:].apply(lambda row: row.nsmallest(2).values[-1],axis=1)

zero_one_gb = whole_grade_books[max-min==1]
all_zero_gb = whole_grade_books[max+min==0]
unique_zero_gb = whole_grade_books[(min==0) & (sec_min>0)]
unique_zero_gb["worst_model"] = unique_zero_gb.iloc[:,4:].idxmin(axis="columns")

zero_one_gb.to_csv("./gradebooks/grade_book_zero_one.csv", index=False)
all_zero_gb.to_csv("./gradebooks/grade_book_all_zero.csv", index=False)
unique_zero_gb.to_csv("./gradebooks/grade_book_unique_zero.csv", index=False)

/var/folders/1s/gdw_sfh525x0796jh_nnmx6r0000gn/T/ipykernel_12487/2293174205.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unique_zero_gb["worst_model"] = unique_zero_gb.iloc[:,4:].idxmin(axis="columns")


In [23]:
unique_zero_gb_with_pred = unique_zero_gb.copy()
zero_one_gb_with_pred = zero_one_gb.copy()
all_zero_gb_with_pred = all_zero_gb.copy()

In [24]:
print (model_preds["multimodal-baselines"]["hotpot"].keys())
que_img = model_preds["multimodal-baselines"]["hotpot"][["question_id", "image_id"]]

Index(['question_id', 'image_id', 'prediction', 'pred_source', 'question',
       'answers', 'question_tokens', 'accu_hotpot'],
      dtype='object')


In [25]:
for k, v in model_preds.items():
    for i in v.keys():
        temp = v[i].rename(columns={"prediction":"pred_"+i, "pred_source":"pred_source_"+i})
        unique_zero_gb_with_pred = unique_zero_gb_with_pred.merge(temp[["question_id", "pred_"+i, "pred_source_"+i]], on="question_id", how="inner")
        zero_one_gb_with_pred = zero_one_gb_with_pred.merge(temp[["question_id", "pred_"+i, "pred_source_"+i]], on="question_id", how="inner")
        all_zero_gb_with_pred = all_zero_gb_with_pred.merge(temp[["question_id", "pred_"+i, "pred_source_"+i]], on="question_id", how="inner")

In [26]:
unique_zero_gb_with_pred = unique_zero_gb_with_pred.merge(que_img, on="question_id", how="inner")
zero_one_gb_with_pred = zero_one_gb_with_pred.merge(que_img, on="question_id", how="inner")
all_zero_gb_with_pred = all_zero_gb_with_pred.merge(que_img, on="question_id", how="inner")

In [27]:
print (unique_zero_gb_with_pred.columns)
print (zero_one_gb_with_pred.columns)
print (all_zero_gb_with_pred.columns)

Index(['question_id', 'question', 'answers', 'question_tokens',
       'accu_hotpot-lorra', 'accu_hotpot-without-mmt',
       'accu_hotpot-without-object-label', 'accu_hotpot', 'accu_object',
       'accu_ocr', 'accu_question', 'accu_lorra', 'accu_m4c', 'accu_tap',
       'worst_model', 'pred_hotpot-lorra', 'pred_source_hotpot-lorra',
       'pred_hotpot-without-mmt', 'pred_source_hotpot-without-mmt',
       'pred_hotpot-without-object-label',
       'pred_source_hotpot-without-object-label', 'pred_hotpot',
       'pred_source_hotpot', 'pred_object', 'pred_source_object', 'pred_ocr',
       'pred_source_ocr', 'pred_question', 'pred_source_question',
       'pred_lorra', 'pred_source_lorra', 'pred_m4c', 'pred_source_m4c',
       'pred_tap', 'pred_source_tap', 'image_id'],
      dtype='object')
Index(['question_id', 'question', 'answers', 'question_tokens',
       'accu_hotpot-lorra', 'accu_hotpot-without-mmt',
       'accu_hotpot-without-object-label', 'accu_hotpot', 'accu_object',
    

In [29]:
print (unique_zero_gb_with_pred.shape)
print (zero_one_gb_with_pred.shape)
print (all_zero_gb_with_pred.shape)
unique_zero_gb_with_pred.to_csv("./gradebooks/grade_book_unique_zero_with_pred.csv", index=False)
zero_one_gb_with_pred.to_csv("./gradebooks/grade_book_zero_one_with_pred.csv", index=False)
all_zero_gb_with_pred.to_csv("./gradebooks/grade_book_all_zero_with_pred.csv", index=False)

(210, 36)
(2803, 35)
(1240, 35)


In [35]:
print (all_zero_gb_with_pred[all_zero_gb_with_pred["question_id"].isin(obj_q)].shape, len(obj_q))
print (all_zero_gb_with_pred[all_zero_gb_with_pred["question_id"].isin(pos_q)].shape, len(pos_q))
print (all_zero_gb_with_pred[all_zero_gb_with_pred["question_id"].isin(text_q)].shape, len(text_q))

(519, 35) 2024
(129, 35) 495
(1213, 35) 4743


In [36]:
hard_question_obj = all_zero_gb_with_pred[all_zero_gb_with_pred["question_id"].isin(obj_q)]
hard_question_pos = all_zero_gb_with_pred[all_zero_gb_with_pred["question_id"].isin(pos_q)]
hard_question_text = all_zero_gb_with_pred[all_zero_gb_with_pred["question_id"].isin(text_q)]

hard_question_obj.to_csv("./gradebooks/grade_book_hard_obj.csv", index=False)
hard_question_pos.to_csv("./gradebooks/grade_book_hard_pos.csv", index=False)
hard_question_text.to_csv("./gradebooks/grade_book_hard_text.csv", index=False)